<a href="https://colab.research.google.com/github/Pt-home/Music-Source-Separation/blob/main/Full_Clean_%26_Align_for_Vocal_%2B_Instrumental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🛠 УСТАНОВКА
!pip install librosa pydub soundfile numpy scipy

In [2]:
# 📁 ІМПОРТ
import librosa
import numpy as np
import soundfile as sf
from scipy.signal import correlate, butter, sosfilt
from google.colab import files
import os

In [3]:
# 🔽 ЗАВАНТАЖЕННЯ
print("Завантажте два аудіофайли: вокал та інструментал (mp3 або wav)")
uploaded = files.upload()

Завантажте два аудіофайли: вокал та інструментал (mp3 або wav)


Saving insts.mp3 to insts.mp3
Saving vocals.mp3 to vocals.mp3


In [5]:
# 🔍 ПОШУК ФАЙЛІВ
def find_audio_file(name_part, extensions=[".wav", ".mp3"]):
    for fname in uploaded:
        if name_part in fname.lower() and any(fname.lower().endswith(ext) for ext in extensions):
            return fname
    return None

vocal_file = find_audio_file("vocal")
inst_file = find_audio_file("inst")

assert vocal_file and inst_file, "❌ Не знайдено обидва файли: vocal + instrumental!"

In [12]:
# 🎵 ЗАВАНТАЖЕННЯ
vocal, sr_vocal = librosa.load(vocal_file, sr=None, mono=True)
inst, sr_inst = librosa.load(inst_file, sr=None, mono=True)
assert sr_vocal == sr_inst, "❌ Sampling rates не співпадають!"

In [13]:
# ✂️ Обрізання
min_len = min(len(vocal), len(inst))
vocal = vocal[:min_len]
inst = inst[:min_len]

# 🎚 Нормалізація
def normalize(audio):
    return audio / np.max(np.abs(audio))

vocal_norm = normalize(vocal)
inst_norm = normalize(inst)

# 🔄 Корекція фази
def align_signals(sig1, sig2):
    correlation = correlate(sig1, sig2, mode='full')
    lag = np.argmax(correlation) - len(sig2) + 1
    if lag > 0:
        sig2 = np.pad(sig2, (lag, 0), mode='constant')[:len(sig1)]
    elif lag < 0:
        sig2 = np.pad(sig2, (0, -lag), mode='constant')[:len(sig1)]
    return sig2, lag

inst_aligned, shift = align_signals(vocal_norm, inst_norm)
print(f"✅ Зсув фази: {shift} семплів")

✅ Зсув фази: -625 семплів


In [14]:
# 💾 ЗБЕРЕЖЕННЯ
sf.write("vocal_corrected.wav", vocal_norm, sr_vocal)
sf.write("instrumental_corrected.wav", inst_aligned, sr_vocal)

In [15]:
# 🧼 ФІЛЬТР ДЛЯ ОЧИСТКИ
def bandpass_filter(audio, sr, lowcut, highcut):
    sos = butter(10, [lowcut, highcut], btype='band', fs=sr, output='sos')
    return sosfilt(sos, audio)

def spectral_denoise(audio, sr, threshold_ratio=0.1):
    S = librosa.stft(audio)
    mag, phase = np.abs(S), np.angle(S)
    mask = mag > (threshold_ratio * np.mean(mag))
    denoised = mag * mask
    return librosa.istft(denoised * np.exp(1j * phase))

# 📦 ОЧИСТКА вокалу
vocal_filtered = bandpass_filter(vocal_norm, sr_vocal, 100, 18000)
vocal_clean = spectral_denoise(vocal_filtered, sr_vocal)
sf.write("vocal_cleaned.wav", vocal_clean, sr_vocal)

# 📦 ОЧИСТКА інструменталу
inst_filtered = bandpass_filter(inst_aligned, sr_vocal, 80, 18000)
inst_clean = spectral_denoise(inst_filtered, sr_vocal)
sf.write("instrumental_cleaned.wav", inst_clean, sr_vocal)

print("✔ Збережено всі 4 файли")

✔ Збережено всі 4 файли


In [16]:
# ⬇️ СКАЧУВАННЯ
files.download("vocal_corrected.wav")
files.download("instrumental_corrected.wav")
files.download("vocal_cleaned.wav")
files.download("instrumental_cleaned.wav")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>